# Fama-French 6 factor Model Analysis Tool

### **references :**
1. https://ssupapers.tistory.com/1
2. ChatGPT

<br>

- 원본 3 factor 코드(PAPERS tistory 블로그 제공) 활용.
- RMSE 도출하는 로직 추가.
- 추가 팩터인 UMD(Up Minus Down, Momentum Factor)도 분석에 포함.

---

# I. 필요 라이브러리 호출 및 함수 정의

In [4]:
import yfinance as yf
import pandas_datareader.data as web
import pandas as pd
import statsmodels.api as sm
import numpy as np


# 월말 수익률 도출 메소드
def get_monthly_returns(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start = start_date, end = end_date, interval = '1mo')

    x = stock_data['Adj Close'].resample('M').last().pct_change().dropna()

    x.name = 'Return' # = 수익률

    return x


# 파마-프렌치 5 factor 데이터 가져오는 메소드
def get_factors(start_date, end_date):
    # pandas datareader를 사용하여 F-F_Research_Data_5_Factors_2x3 데이터를 가져옴, 딕셔너리 형태이므로 첫 번째 데이터프레임을 추출
    ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start_date, end_date)[0]

    # PeriodIndex를 Timestamp로 변환
    ff_data.index = ff_data.index.to_timestamp()

    # 필요한 컬럼만 선택하여 월 단위로 리샘플링
    ff_data = ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']].resample('M').last()

    # 퍼센트 단위를 소수점으로 변환
    ff_data /= 100

    return ff_data


# 파마-프랜치 6번째 factor인 모멘텀 데이터 가져오는 메소드
def get_momentum(start_date, end_date):
    # pandas datareader를 사용하여 F-F_Momentum_Factor 데이터를 가져옴, 딕셔너리 형태이므로 첫 번째 데이터프레임을 추출
    momentum = web.DataReader('F-F_Momentum_Factor', 'famafrench', start_date, end_date)[0]

    # PeriodIndex를 Timestamp로 변환
    momentum.index = momentum.index.to_timestamp()

    # 월 단위로 리샘플링
    momentum = momentum.resample('M').last()

    # 퍼센트 단위를 소수점으로 변환
    momentum /= 100

    momentum.columns = ['UMD']

    return momentum


# 수익률과 팩터 데이터를 하나의 데이터프레임으로 병합하는 메소드
def merge_data(ticker, start_date, end_date):
    monthly_returns = get_monthly_returns(ticker, start_date, end_date)

    fama5 = get_factors(start_date, end_date)

    momentum = get_momentum(start_date, end_date)

    factors = pd.merge(fama5, momentum, left_index = True, right_index = True)

    df = pd.merge(monthly_returns, factors, left_index = True, right_index = True)

    df['Excess Return'] = df['Return'] - df['RF'] # Excess Return : 초과수익률 의미

    return df


# 파마-프렌치 6 factor 모델로 OLS 회귀를 진행하여 타겟변수 ETF에 대해 분석하는 메소드
def estimate_ff_model(df):
    X = df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']]

    # 독립변수 X : 5 factors
    X = sm.add_constant(X) # 모델에 상수항(alpha = y절편)을 추가

    # 종속변수 y : 분석하고자 하는 ETF의 (무위험채권 대비) 초과수익률
    y = df['Excess Return']

    # OLS(최소자승법)을 사용해 회귀모델 적합
    model = sm.OLS(y, X).fit()

    # 예측값 계산
    predictions = model.predict(X)

    # RMSE 계산
    rmse = np.sqrt(((predictions - y) ** 2).mean())

    return model.summary(), rmse

---

# II. 각 ETF 분석 수행
- User에게 직접 ETF ticker명 이름을 input으로 받도록 수정.


In [ ]:
def main():
    ticker_list = ['IWM', 'SPY', 'VTV', 'VUG', 'MTUM', 'QUAL', 'VYMI', 'USMV', 'KBE', 'IYK', 'IYC']

    start_date = '2000-01-01'
    end_date = '2024-09-01'

    for ticker in ticker_list:
        df = merge_data(ticker, start_date, end_date)

        model_summary, rmse = estimate_ff_model(df)

        print(f'\n{model_summary}\n') ; print(f'\nRMSE : {rmse}\n')

if __name__ == "__main__":
    main()

| ETF | RMSE | R-squared | Adjusted R-squared | Constant | Mkt-RF | SMB | HML | RMW | CMA | UMD |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| IWM | 0.00681 | 0.986 | 0.986 | -0.0009 | 1.0099 | 0.8192 | 0.0858 | -0.0868 | -0.0056 | 0.0378 |
| SPY | 0.00497 | 0.988 | 0.988 | -0.0002 | 0.9939 | -0.1570 | 0.0119 | 0.0605 | 0.0416 | -0.0416 |
| VTV | 0.00900 | 0.957 | 0.956 | -0.0004 | 0.9278 | -0.1493 | 0.2990 | 0.0544 | 0.1570 | -0.0349 |
| VUG | 0.00817 | 0.971 | 0.970 | 0.0003 | 1.0785 | -0.1204 | -0.2542 | 0.0251 | -0.1495 | -0.0099 |
| MTUM | 0.01390 | 0.906 | 0.902 | -0.0008 | 1.0747 | -0.1237 | -0.0335 | -0.1057 | 0.0343 | 0.3843 |
| QUAL | 0.00730 | 0.973 | 0.972 | -0.0007 | 1.0048 | -0.1225 | -0.0175 | 0.1897 | -0.0302 | 0.0074 |
| VYMI | 0.02178 | 0.785 | 0.771 | -0.0021 | 0.7684 | -0.1774 | 0.3093 | -0.0583 | 0.1839 | -0.2281 |
| USMV | 0.01320 | 0.847 | 0.841 | -0.0007 | 0.7846 | -0.0958 | -0.0632 | 0.2884 | 0.2260 | 0.1263 |
| KBE | 0.03402 | 0.819 | 0.814 | 0.0013 | 0.9012 | 0.1736 | 1.4094 | -0.6084 | -0.7755 | -0.1535 |
| IYK | 0.02021 | 0.717 | 0.711 | -0.0011 | 0.7678 | -0.0484 | -0.0464 | 0.4134 | 0.4817 | -0.0093 |
| IYC | 0.01852 | 0.870 | 0.868 | -0.0004 | 1.0036 | 0.1468 | -0.0648 | 0.1790 | -0.0695 | -0.0833 |